In [1]:
# 라이브러리 임포트

import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
# 버전 확인

print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.1.3
Numpy : 1.19.2
Scikit-Learn : 0.23.2
Python 3.8.5


In [3]:
df=pd.read_csv('project2/Card.csv')

In [4]:
df.shape

(24697792, 12)

In [18]:
print(len(df['CARD_CCG_NM'].unique()))
df['CARD_CCG_NM'].unique()

227


array(['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시',
       '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군',
       '가평군', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '광명시', '광주시',
       '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시 분당구', '성남시 수정구',
       '성남시 중원구', '수원시 권선구', '수원시 영통구', '수원시 장안구', '수원시 팔달구', '시흥시',
       '안산시 단원구', '안산시 상록구', '안성시', '안양시 동안구', '안양시 만안구', '양주시', '양평군',
       '여주시', '연천군', '오산시', '용인시 기흥구', '용인시 수지구', '용인시 처인구', '의왕시',
       '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '거제시', '거창군',
       '김해시', '남해군', '밀양시', '사천시', '산청군', '양산시', '의령군', '진주시', '창녕군',
       '창원시 마산합포구', '창원시 마산회원구', '창원시 성산구', '창원시 의창구', '창원시 진해구', '통영시',
       '하동군', '함안군', '함양군', '합천군', '경산시', '경주시', '고령군', '구미시', '군위군',
       '김천시', '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시',
       '영천시', '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '칠곡군', '포항시 남구',
       '포항시 북구', '광산구', '남구', '동구', '북구', '서구', '달서구', '달성군', '수성구', '중구',
      

In [16]:
df=df.fillna('세종시')

In [17]:
df.isnull().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

In [20]:
# 월, 년 추출하기 위한 사용자 정의 함수 생성
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [21]:
df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 13 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   CARD_SIDO_NM  object
 1   CARD_CCG_NM   object
 2   STD_CLSS_NM   object
 3   HOM_SIDO_NM   object
 4   HOM_CCG_NM    object
 5   AGE           object
 6   SEX_CTGO_CD   int64 
 7   FLC           int64 
 8   CSTMR_CNT     int64 
 9   AMT           int64 
 10  CNT           int64 
 11  year          int64 
 12  month         int64 
dtypes: int64(7), object(6)
memory usage: 2.4+ GB


In [24]:
df = df.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

In [26]:
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df=df.groupby(columns).sum().reset_index(drop=False)

In [27]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [28]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [29]:
k = int(len(x)*0.9)

In [30]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [34]:
import lightgbm as lgb

In [35]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [40]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [44]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045616 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 107
[LightGBM] [Info] Number of data points in the train set: 951654, number of used features: 8
[LightGBM] [Info] Start training from score 2.657378
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.98152
[200]	valid_0's rmse: 1.81073
[300]	valid_0's rmse: 1.68877
[400]	valid_0's rmse: 1.6029
[500]	valid_0's rmse: 1.51398
[600]	valid_0's rmse: 1.44854
[700]	valid_0's rmse: 1.39655
[800]	valid_0's rmse: 1.35321
[900]	valid_0's rmse: 1.31575
[1000]	valid_0's rmse: 1.2826
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.2826
